In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"gglavanya","key":"bcb9fc307f691f82bff71a6b448a6758"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!pip install kaggle


In [5]:
!kaggle datasets download -d mirichoi0218/insurance
!unzip -q insurance.zip



Dataset URL: https://www.kaggle.com/datasets/mirichoi0218/insurance
License(s): DbCL-1.0
  0% 0.00/16.0k [00:00<?, ?B/s]
100% 16.0k/16.0k [00:00<00:00, 86.8MB/s]


In [6]:
import pandas as pd
df=pd.read_csv("insurance.csv")

In [23]:
df.head()
df.tail()
df.info()
df.describe()
df.isnull().sum()
df.duplicated().sum()
df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


,0
age,int64
sex,object
bmi,float64
children,int64
smoker,object
region,object
charges,float64


In [26]:
df.isnull().sum()
df.dropna()


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [27]:
df=pd.get_dummies(df,columns=["sex","region","smoker"], drop_first=True)
df.head()

,age,bmi,children,charges,sex_male,region_northwest,region_southeast,region_southwest,smoker_yes
0,19,27.900,0,16884.92400,False,False,False,True,True
1,18,33.770,1,1725.55230,True,False,True,False,False
2,28,33.000,3,4449.46200,True,False,True,False,False
3,33,22.705,0,21984.47061,True,True,False,False,False
4,32,28.880,0,3866.85520,True,True,False,False,False


In [31]:
from copy import copy
from sklearn.preprocessing import StandardScaler
numerical_cols=['age','bmi','children','charges']
scaler=StandardScaler()
df[numerical_cols]=scaler.fit_transform(df[numerical_cols])

df[numerical_cols].describe()

,age,bmi,children,charges
count,1.338000e+03,1.338000e+03,1.338000e+03,1.338000e+03
mean,-1.805565e-16,-2.124194e-16,-5.576008e-17,-8.098488e-17
std,1.000374e+00,1.000374e+00,1.000374e+00,1.000374e+00
min,-1.509965e+00,-2.412011e+00,-9.086137e-01,-1.003557e+00
25%,-8.691547e-01,-7.164063e-01,-9.086137e-01,-7.046504e-01
50%,-1.474046e-02,-4.320880e-02,-7.876719e-02,-3.212089e-01
75%,8.396738e-01,6.611572e-01,7.510793e-01,2.783441e-01
max,1.765289e+00,3.685522e+00,3.240619e+00,4.171663e+00


In [45]:
from sklearn.model_selection import train_test_split
x=df.drop('charges',axis=1)
y=df['charges']
X_train, X_test, Y_train,Y_test=train_test_split(x,y,test_size=0.2,random_state=42)
print("Training set shape:", X_train.shape, Y_train.shape)
print("Test set shape:", X_test.shape, Y_test.shape)

Training set shape: (1070, 8) (1070,)
Test set shape: (268, 8) (268,)


In [46]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
model=LinearRegression()
model.fit(X_train,Y_train)

LinearRegression()

In [38]:
Y_pred=model.predict(X_test)
mse=mean_squared_error(Y_test,Y_pred)

In [39]:
root_mse=mse**0.5
print("Root Mean Squared Error:",root_mse)


Root Mean Squared Error: 0.4788147414975719


In [40]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
model=LinearRegression()
scores=cross_val_score(model,x,y,cv=5,scoring='neg_mean_squared_error')
mse_scores=-scores
rmse_scores=mse_scores**0.5


print("Training set shape:", X_train.shape, Y_train.shape)
print("Test set shape:", X_test.shape, Y_test.shape)

Training set shape: (1070, 8) (1070,)
Test set shape: (268, 8) (268,)


In [49]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Example input: a 30-year-old male, BMI=25, 1 child, smoker, lives in southeast region
new_data = pd.DataFrame({
    'age': [30],
    'bmi': [25],
    'children': [1],
    'sex_male': [1],
    'smoker_yes': [1],
    'region_northwest': [0],
    'region_southeast': [1],
    'region_southwest': [0]
})

# Assuming you scaled numerical columns in your training data using `scaler`
numerical_cols = ['age', 'bmi', 'children']

# Fit the scaler on the training data and then transform the new data
scaler = StandardScaler()
scaler.fit(X_train[numerical_cols]) # Fit on the training data
new_data[numerical_cols] = scaler.transform(new_data[numerical_cols]) # Transform new data

# Ensure the columns in new_data match the order of columns in X_train
new_data = new_data[X_train.columns]

predicted_charge = model.predict(new_data)
print(f"Predicted insurance charge: ${predicted_charge[0]:.2f}")

Predicted insurance charge: $14.79
